In [2]:
%load_ext autoreload
%autoreload 2

import pennylane as qml
import math
from pennylane import qaoa
from pennylane import numpy as np
import matplotlib.pyplot as plt
from vqa.hamiltonian.protein_folding import protein_folding_hamiltonian
from vqa.hamiltonian.Protein_models.CoordinateBased_HPLattice import CoordinateBased_HPLattice
from vqa.utils.protein_utils import *
from collections import Counter
import pandas as pd
import csv
import os.path
import scipy
from scipy.optimize import minimize

In [3]:
# Make an instance for the protein
sequence = [1,0,0,1] #HPPH
L1 = 2
L2 = 2

lambda_vector = (2.1, 2.4, 3)

protein = CoordinateBased_HPLattice((L1, L2), sequence = sequence, lambda_vector = lambda_vector)
print(protein)
protein.calc_solution_sets()
feasible_set = np.array(protein.feasible_set)


O:
[-2.1, -2.1, -2.1, -2.1, -2.1, -2.1, -2.1, -2.1]
T:
[[ 0.   4.2  0.   0.   2.4  0.  -1.  -1. ]
 [ 4.2  0.   0.   0.   0.   2.4 -1.  -1. ]
 [ 0.   0.   0.   4.2  0.   0.   2.4  0. ]
 [ 0.   0.   4.2  0.   0.   0.   0.   2.4]
 [ 2.4  0.   0.   0.   0.   4.2  0.   0. ]
 [ 0.   2.4  0.   0.   4.2  0.   0.   0. ]
 [-1.  -1.   2.4  0.   0.   0.   0.   4.2]
 [-1.  -1.   0.   2.4  0.   0.   4.2  0. ]]
Dn:
[2, 2, 2, 2]


In [4]:
# Make into Hamiltonian
H_cost = protein_folding_hamiltonian(protein)
num_qubits = protein.num_bits
qubits = range(protein.num_bits)

H_mixer = qaoa.x_mixer(qubits)

In [39]:
dev = qml.device('default.qubit', wires = qubits)

# Put the Mixer and Cost Hamiltonians in a layer so that we can repeat it
def qaoa_layer(gamma, beta):
    qaoa.cost_layer(gamma, H_cost)
    qaoa.mixer_layer(beta, H_mixer)
    
# Then repeat it in a circuit with starting in a super position of all bitstrings
def circuit(params): # Gamma and Beta values can be put together to be an array of parameters
    for q in qubits:     # To start in a superposition we place a Hadamard on all qubits
        qml.Hadamard(wires = q)
    qml.layer(qaoa_layer, len(params[0]), params[0], params[1])
    
# Obtains the probability for all the states
@qml.qnode(dev)
def get_probs(params):
    circuit(params)
    return qml.probs(wires = qubits)

def CVaRsamp(params, alpha=0.3, n=100):#n=antal samples
    circuit(params)
    
    probs = get_probs(params)  # hämta sannolikhetsfördelningen (matris, första värdet är p(00000000), andra är p(00000001) osv)
    
    index_samples= np.random.choice(np.arange(len(probs), dtype=int), size=n, replace=True,p=probs)  # tar n samples av probs, ger skum lista med index motsvarande konfiguration (index 0 är tillståndet 00000000, index 1 =00000001 osv)
    energy_of_samples=energies_of_set(protein.get_solution_set(), H_cost,8) [index_samples]  #ger en lista där index i index_samples är utbytta mot deras motsvarande energi. 
    sorted_energy_samples=sort_over_threshhold(energy_of_samples,-10)[0] #sorterar hela energilistan
    K=int(alpha*n) #antal samples att ta väntevärde över. 
    summa=sum(sorted_energy_samples[:K]) #summera de K minsta energierna.
    expvalue=np.float64(summa/K)
    
    return expvalue

In [53]:
def cost_execution(params):
    params = np.reshape(params, (2, 2))

    global iterations

    cost = CVaRsamp(params, n=num_of_samples)

    #if iterations % 50 == 0:
    print("Cost at Step {}: {}".format(iterations, cost))

    iterations += 1
    return cost

In [59]:
iterations = 0
p=2
counter = 0
j=0
num_of_samples=50

initial_params=np.array([[0.1, 0.1], [0.5, 0.5]], requires_grad = True)
print(initial_params.shape)
out = minimize(cost_execution, x0=initial_params, method="COBYLA", options={"maxiter": 10}, tol=0.0001)
out_params = out["x"]
print(out["x"])
print(out["success"])
probs = get_probs(out_params)

ground_energy, ground_states_i = get_ground_states_i(feasible_set, H_cost) # get the ground states

#if np.sum(probs[ground_states_i]) < 0.05:
counter+=1
print('Try ' + str(j) + ': Success probability of training: ', np.sum(probs[ground_states_i]))
print('For: ' + str(num_of_samples) + ' samples, the prob of measuring any groundstate was at least 5% : ' + str(j-counter) + ' times out of ' + str(j) + ' tries.')
print("="*20)

(2, 2)
Cost at Step 0: -1.693333333333333
Cost at Step 1: -2.2466666666666675
Cost at Step 2: -4.08


/var/folders/cw/p649yvnd6tl_8jn4s2cchc7m0000gn/T/ipykernel_6415/520255164.py:9: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  out = minimize(cost_execution, x0=initial_params, method="COBYLA", options={"maxiter": 10}, tol=0.0001)


Cost at Step 3: -1.9066666666666665
Cost at Step 4: -2.9866666666666664
Cost at Step 5: -2.56
Cost at Step 6: -2.106666666666667
Cost at Step 7: -3.6466666666666674
Cost at Step 8: -2.326666666666667
Cost at Step 9: -4.153333333333333
[[1.07100363 0.33422393]
 [1.57623719 0.46859535]]
False
Try 0: Success probability of training:  0.1338524039455679
For: 50 samples, the prob of measuring any groundstate was at least 5% : -1 times out of 0 tries.
